In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from alphagogoat.delphox import Delphox
from alphagogoat.utils import DataExtractor
# from alphagogoat.embedder import process_battle
from pathlib import Path
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import json
import numpy as np
import json
import logging
import copy

from poke_env.environment.battle import Battle
from poke_env.environment.pokemon import Pokemon
from poke_env.environment.effect import Effect
import torch


# from alphagogoat.pokedex import make_pokedex


LOGGER = logging.getLogger('poke-env')
GEN = 8
# POKEDEX = make_pokedex()

In [17]:
def process_battle(battle_json: str) -> list[Battle]:
    with open(battle_json) as f:
        battle_data = json.load(f)
    history = battle_data['log'].split('\n')
    curr_battle = Battle(battle_data['id'], battle_data['p1'], LOGGER, GEN)
    curr_battle._opponent_username = battle_data['p2']
    battle_objects = []
    for line in history:
        try:
            curr_battle._parse_message(line.split('|'))
            if line.split('|')[1] == 'turn':
                battle_objects.append(copy.deepcopy(curr_battle))
        except:
            continue

    return battle_objects


In [6]:
battle_paths = [f"cache/replays/{file.name}" for file in Path("cache/replays").iterdir() if file.name.endswith(".json")]

def get_turns(battle_json_path):
    return DataExtractor(battle_json_path)

# DEBUGGING
battle_paths = battle_paths[:5]
extractors = Parallel(n_jobs=4)(delayed(get_turns)(battle_json_path) for battle_json_path in tqdm(battle_paths))

  0%|          | 0/5 [00:00<?, ?it/s]

In [7]:
pokedex = {}
for filepath in tqdm(list(Path("cache/teams").iterdir())):
    with open(filepath, 'r') as f:
        data = json.load(f)
    for key, value in data.items():
        if key not in pokedex:
            pokedex[key] = {k: [] for k in value.keys()}
        for k, v in value.items():
            pokedex[key][k].append(v)

for pokemon, dicts in tqdm(pokedex.items()):
    pokedex[pokemon]['level'] = int(np.round(np.mean(pokedex[pokemon]['level'])))
    for k1 in ['abilities', 'moves', 'items']:
        d = {}
        if k1 in dicts:
            for dict_ in dicts[k1]:
                if not isinstance(dict_, dict):
                    continue
                for k2, prob in dict_.items():
                    if k2 not in d:
                        d[k2] = []
                    d[k2].append(prob)
            for k2, probs in d.items():
                d[k2] = np.mean(probs)
        pokedex[pokemon][k1] = d

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/460 [00:00<?, ?it/s]

In [8]:
pokedex['Pikachu']

{'level': 92,
 'abilities': {'Lightning Rod': 1.0},
 'items': {'Light Ball': 1.0},
 'moves': {'Volt Tackle': 1.0,
  'Volt Switch': 0.7812379310344825,
  'Surf': 0.7469724137931036,
  'Knock Off': 0.7469655172413794,
  'Iron Tail': 0.7248241379310345}}

In [9]:
max_abilities = 0
max_items = 0
max_moves = 0
for pokemon, data in pokedex.items():
    max_abilities = max(len(data['abilities']), max_abilities)
    max_moves = max(len(data['moves']), max_moves)
    max_items = max(len(data['items']), max_items)
print(f"{max_abilities=}\n{max_items=}\n{max_moves=}")

max_abilities=3
max_items=6
max_moves=8


In [10]:
turns = extractors[0].turns
extractor = extractors[0]
embedding = extractor.embed(turns[3])
embedding.shape

successfully extracted all features


torch.Size([4981])

In [18]:
battle_json = "cache/replays/gen8randombattle-1123651831.json"
turns = process_battle(battle_json)

In [20]:
team = {}
for turn in turns:
    active = turn.active_pokemon
    team[active.species] = active
    print(active.boosts)

print(team['incineroar'].base_stats)

{'accuracy': 0, 'atk': 0, 'def': 0, 'evasion': 0, 'spa': 0, 'spd': 0, 'spe': 0}
{'accuracy': 0, 'atk': 0, 'def': 0, 'evasion': 0, 'spa': 0, 'spd': 0, 'spe': 0}
{'accuracy': 0, 'atk': 0, 'def': 0, 'evasion': 0, 'spa': 0, 'spd': 0, 'spe': 0}
{'accuracy': 0, 'atk': 0, 'def': 0, 'evasion': 0, 'spa': 0, 'spd': 0, 'spe': 0}
{'accuracy': 0, 'atk': 0, 'def': 0, 'evasion': 0, 'spa': 0, 'spd': 0, 'spe': 0}
{'accuracy': 0, 'atk': 0, 'def': 0, 'evasion': 0, 'spa': 0, 'spd': 0, 'spe': 0}
{'accuracy': 0, 'atk': 0, 'def': 0, 'evasion': 0, 'spa': 0, 'spd': 0, 'spe': 0}
{'accuracy': 0, 'atk': 0, 'def': 0, 'evasion': 0, 'spa': 0, 'spd': 0, 'spe': 0}
{'accuracy': 0, 'atk': 0, 'def': 0, 'evasion': 0, 'spa': 0, 'spd': 0, 'spe': 0}
{'accuracy': 0, 'atk': 0, 'def': 0, 'evasion': 0, 'spa': 0, 'spd': 0, 'spe': 0}
{'accuracy': 0, 'atk': 0, 'def': 0, 'evasion': 0, 'spa': 0, 'spd': 0, 'spe': 0}
{'accuracy': 0, 'atk': 0, 'def': 0, 'evasion': 0, 'spa': 0, 'spd': 0, 'spe': 0}
{'accuracy': 0, 'atk': 0, 'def': 0, 'eva

In [ ]:
pokedex['Incineroar'].keys()